## Time Series of data

In [1]:
%load_ext pypath_magic

In [13]:
%pypath -a ../utils/

Added u'/home/critechuser/REPOS/StormS/utils' to path.


In [14]:
%pypath -a ../src/

Added u'/home/critechuser/REPOS/StormS/src' to path.


In [170]:
#Python modules
import numpy as np
from matplotlib import animation,rc
import matplotlib.pyplot as plt
from netCDF4 import Dataset
from mpl_toolkits.basemap import Basemap, shiftgrid
from IPython.display import HTML
import subprocess

In [16]:
##local modules
from grid import *
from dep import *
import glob
import mdf

In [17]:
# clean up 
%pypath -d ../src/
%pypath -d ../utils/

Deleted u'/home/critechuser/REPOS/StormS/src' from path
Deleted u'/home/critechuser/REPOS/StormS/utils' from path


In [6]:
rc('animation',html='html5')

In [123]:
PATH='../ss_test/20170227.00/'
path0='../ss_test/'

In [125]:
basename='tide'

In [126]:
grid=Grid.fromfile(PATH+basename+'.grd')
lon=grid.x[0,:].data
lat=grid.y[:,0].data

In [127]:
deb=Dep.read(PATH+basename+'.dep',grid.shape)
b=deb.val
w=np.isnan(b)

In [128]:
m = Basemap(projection='cyl',llcrnrlat=lat.min(),urcrnrlat=lat.max(),\
             llcrnrlon=lon.min(),urcrnrlon=lon.max(),resolution='h')


In [129]:
# define parallels and meridians to draw.
parallels = np.arange(-90.,90,20.)
meridians = np.arange(0.,360.,20.)



Read data

In [130]:
data=Dataset(PATH+'trim-{}.nc'.format(basename))

In [131]:
inp, order = mdf.read(PATH+basename+'.mdf')

In [132]:
idate=datetime.datetime.strptime(inp['Itdate'],'%Y-%m-%d')

In [133]:
tstart=idate+datetime.timedelta(minutes=np.int(inp['Tstart'][0]))

In [134]:
h=data['S1'][:]
time=data['time'][:]
tw=[]
for l in time : tw.append(tstart+datetime.timedelta(0,int(l)))
tcw=np.array(tw).flatten()

In [135]:
iframes=h.shape[0]
fig = plt.figure(figsize=(10,8))
ax = fig.add_subplot(111)
bh=np.ma.masked_where(w==True,h[0,:,:].T)
#H=ax.imshow(np.flipud(bh),animated=True, vmin=h.min(), vmax=h.max())
H=m.imshow(bh,animated=True, vmin=h.min(), vmax=h.max())
plt.colorbar(H, orientation='horizontal')
date_text=fig.text(0.1,0.95, datetime.datetime.strftime(tcw[0],'%a %b %d  %H:%M:%S %Z %Y' ))
# draw coastlines, parallels, meridians.
m.drawcoastlines(linewidth=1.5)
m.drawparallels(parallels,labels=[False,True,True,False])
m.drawmeridians(meridians,labels=[True,False,False,True])


/usr/local/lib/python2.7/dist-packages/matplotlib/pyplot.py:516: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)


{-40.0: ([<matplotlib.lines.Line2D at 0x7f328cadf590>],
  [<matplotlib.text.Text at 0x7f328ca1d9d0>]),
 -20.0: ([<matplotlib.lines.Line2D at 0x7f328ca01150>],
  [<matplotlib.text.Text at 0x7f328ca1de50>]),
 0.0: ([<matplotlib.lines.Line2D at 0x7f328ca01790>],
  [<matplotlib.text.Text at 0x7f328c97e2d0>]),
 20.0: ([<matplotlib.lines.Line2D at 0x7f328ca01dd0>],
  [<matplotlib.text.Text at 0x7f328c97e710>]),
 40.0: ([<matplotlib.lines.Line2D at 0x7f328ca1d450>],
  [<matplotlib.text.Text at 0x7f328c97eb50>])}

In [136]:
def animate(i):
    bh=np.ma.masked_where(w==True,h[i,:,:].T)    

#    H.set_array(np.flipud(h[i,:,:].T))
    H.set_array(bh)
    date_text.set_text(datetime.datetime.strftime(tcw[i],'%a %b %d  %H:%M:%S %Z %Y' ))
    return H

In [137]:
# call the animator.  blit=True means only re-draw the parts that have changed.
anim = animation.FuncAnimation(fig, animate, frames=iframes, interval=200, blit=False, repeat=False)

In [138]:
#  display the animation
anim

In [14]:
#save animation
#anim.save('tmp/webtest.mp4', fps=10, extra_args=['-vcodec','libx264'])

### Merging the 12h forecasts for longer time span

In [139]:
tstart=datetime.datetime(2017,2,27,0)

In [153]:
tend=datetime.datetime(2017,2,28,0)

In [154]:
dt=(tend-tstart).total_seconds()
ndt=dt/(3600*12)
ndt=np.int(ndt)+1

In [155]:
hmin=-.5
hmax=2.

In [156]:
idate=tstart
path=path0+'{}{:02d}{:02d}.{:02d}/'.format(idate.year,idate.month,idate.day,idate.hour)
d=Dataset(path+'trim-{}.nc'.format(basename))
h = d['S1'][:12,:,:]
time=d['time'][:12]
tw=[]
for l in time : tw.append(tstart+datetime.timedelta(0,int(l)))
tcw=np.array(tw).flatten()
iframes=h.shape[0]
# create initial figure
fig = plt.figure(figsize=(10,8))
ax = fig.add_subplot(111)
bh=np.ma.masked_where(w==True,h[0,:,:].T)
H=m.imshow(bh,animated=True, vmin=hmin, vmax=hmax)
plt.colorbar(H, orientation='horizontal')
date_text=fig.text(0.1,0.95, datetime.datetime.strftime(tcw[0],'%a %b %d  %H:%M:%S %Z %Y' ))
# draw coastlines, parallels, meridians.
m.drawcoastlines(linewidth=1.5)
m.drawparallels(parallels,labels=[False,True,True,False])
m.drawmeridians(meridians,labels=[True,False,False,True])


{-40.0: ([<matplotlib.lines.Line2D at 0x7f328c268fd0>],
  [<matplotlib.text.Text at 0x7f328c07dc90>]),
 -20.0: ([<matplotlib.lines.Line2D at 0x7f328c1a0410>],
  [<matplotlib.text.Text at 0x7f328c09c150>]),
 0.0: ([<matplotlib.lines.Line2D at 0x7f328c1a0a50>],
  [<matplotlib.text.Text at 0x7f328c09c590>]),
 20.0: ([<matplotlib.lines.Line2D at 0x7f328c1a0fd0>],
  [<matplotlib.text.Text at 0x7f328c09c9d0>]),
 40.0: ([<matplotlib.lines.Line2D at 0x7f328c07d710>],
  [<matplotlib.text.Text at 0x7f328c09ce10>])}

In [157]:
# call the animator.  blit=True means only re-draw the parts that have changed.
anim = animation.FuncAnimation(fig, animate, frames=iframes, interval=200, blit=False, repeat=False)

In [158]:
#anim

In [159]:
#save animation
anim.save('tmp/movie1.mp4', fps=10, extra_args=['-vcodec','libx264','-pix_fmt', 'yuv420p'])

In [167]:
#create a list file for the merging below
with open('tmp/movielist', 'w') as f:
    f.write("file movie1.mp4\n")

Create temp files and merge in order to avoid heavy load 

In [168]:
for it in range(1,ndt):
    idate=tstart+datetime.timedelta(hours=12*it)
    path=path0+'{}{:02d}{:02d}.{:02d}/'.format(idate.year,idate.month,idate.day,idate.hour)
    d=Dataset(path+'trim-{}.nc'.format(basename))
    h = d['S1'][:12,:,:]
    time=d['time'][:12]
    tw=[]
    for l in time : tw.append(tstart+datetime.timedelta(0,int(l)))
    tcw=np.array(tw).flatten()
    iframes=h.shape[0]
    anim = animation.FuncAnimation(fig, animate, frames=12, interval=200, blit=False, repeat=False)
    anim.save('tmp/movie{}.mp4'.format(it+1), fps=10, extra_args=['-vcodec','libx264','-pix_fmt', 'yuv420p'])
    with open('tmp/movielist', 'a') as f:
        f.write("file movie{}.mp4\n".format(it+1))

In [171]:
## merge mp4
ex=subprocess.Popen('ffmpeg -f concat -i movielist -c copy movie.mp4', cwd='tmp/', shell=True, stdout=subprocess.PIPE, bufsize=1)